VinDr-Mammo-finding

In [ ]:
import pandas as pd
import numpy as np
import os
import pydicom
import cv2
from tqdm import tqdm
import ast
from pydicom.pixel_data_handlers.util import apply_voi_lut

def fit_image(fname):
    dicom = pydicom.dcmread(fname)
    X = apply_voi_lut(dicom.pixel_array, dicom, prefer_lut=False)
    X = (X - X.min()) / (X.max() - X.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        X = 1 - X
    X = X * 255
    return X

def crop_image(X, xmin, ymin, xmax, ymax):
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    xmax = min(X.shape[1], xmax)
    ymax = min(X.shape[0], ymax)
    return X[ymin:ymax, xmin:xmax]

# 读取CSV文件
csv_path = '/Volumes/图图/VinDr/VinDr-Mammo/finding_annotations.csv'
split_csv_path = '../classification_data/classification_split.csv'
df = pd.read_csv(csv_path)

# 读取数据划分CSV文件
split_df = pd.read_csv(split_csv_path)
split_df = split_df[split_df['dataset'] == 'VinDr-Mammo-finding']

# 修正边界框坐标，筛选有异常的图像
df['finding_categories'] = df['finding_categories'].apply(ast.literal_eval)
df = df[df['finding_categories'].apply(lambda x: x != ['No Finding'])]

# 按 image_id 分组
grouped_df = df.groupby('image_id')

# 定义基本路径
base_path = '/Volumes/图图/VinDr/VinDr-Mammo/images'
target_path = '../classification_data/VinDr-Mammo-finding'

# 创建目标文件夹
os.makedirs(target_path, exist_ok=True)

# 处理每个图像及其对应的边界框
for image_id, group in tqdm(grouped_df):
    study_id = group['study_id'].iloc[0]
    img_dir = os.path.join(base_path, study_id)
    img_path = os.path.join(img_dir, image_id + '.dicom')

    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue

    img = fit_image(img_path)
    
    for i, row in enumerate(group.itertuples()):
        # 生成data_name
        data_name = f"{image_id}_{i}"
        
        # 查找对应的data_split
        split_info = split_df[split_df['data_name'] == data_name]
        if split_info.empty:
            print(f"No split info found for {data_name}")
            continue
            
        data_split = split_info['data_split'].values[0]
        
        # 裁剪图像
        xmin, ymin, xmax, ymax = int(row.xmin), int(row.ymin), int(row.xmax), int(row.ymax)
        cropped_img = crop_image(img, xmin, ymin, xmax, ymax)
        
        # 创建输出目录
        img_folder = os.path.join(target_path, data_split, data_name)
        os.makedirs(img_folder, exist_ok=True)
        
        # 保存图像
        jpg_path = os.path.join(img_folder, 'img.jpg')
        cv2.imwrite(jpg_path, cropped_img)

        # 保存元数据
        info_dict = {
            'Finding': row.finding_categories
        }
        npy_path = os.path.join(img_folder, 'info_dict.npy')
        np.save(npy_path, info_dict)

        print(f'Processed {data_name} for {data_split} set')

print("Processing complete.")

  0%|          | 1/1768 [00:00<20:21,  1.45it/s]

Image and info_dict saved for 001ade2a3cb53fd808bd2856a0df5413_0 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/001ade2a3cb53fd808bd2856a0df5413_0
Image and info_dict saved for 001ade2a3cb53fd808bd2856a0df5413_1 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/001ade2a3cb53fd808bd2856a0df5413_1
Image and info_dict saved for 001ade2a3cb53fd808bd2856a0df5413_2 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/001ade2a3cb53fd808bd2856a0df5413_2


  0%|          | 2/1768 [00:01<20:28,  1.44it/s]

Image and info_dict saved for 002074412a8fc178c271fb93b55c3e29_0 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/002074412a8fc178c271fb93b55c3e29_0


  0%|          | 3/1768 [00:01<16:59,  1.73it/s]

...

Image and info_dict saved for ffb94da823c87898c7ce19b197dc24c9_0 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/ffb94da823c87898c7ce19b197dc24c9_0


100%|█████████▉| 1767/1768 [15:52<00:00,  1.95it/s]

Image and info_dict saved for ffd951bc4cd3d91f8994d2d1d36f038d_0 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Eval/ffd951bc4cd3d91f8994d2d1d36f038d_0


100%|██████████| 1768/1768 [15:53<00:00,  1.85it/s]

Image and info_dict saved for ffe25766ac37ab145ae7f0bd78665f81_0 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/ffe25766ac37ab145ae7f0bd78665f81_0
Image and info_dict saved for ffe25766ac37ab145ae7f0bd78665f81_1 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/ffe25766ac37ab145ae7f0bd78665f81_1
Image and info_dict saved for ffe25766ac37ab145ae7f0bd78665f81_2 in /Volumes/Newsmy/FM_dataset/VinDr-Mammo-finding/Train/ffe25766ac37ab145ae7f0bd78665f81_2
Processing complete.


Detection

In [ ]:
import pandas as pd
import numpy as np
import os
import pydicom
import cv2
from tqdm import tqdm
import ast
from pydicom.pixel_data_handlers.util import apply_voi_lut

def fit_image(fname):
    dicom = pydicom.dcmread(fname)
    X = apply_voi_lut(dicom.pixel_array, dicom, prefer_lut=False)
    
    X = (X - X.min()) / (X.max() - X.min())
    
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        X = 1 - X
    
    X = X * 255
    X = X[10:-10, 10:-10]  # Cut off narrow frame
    
    output = cv2.connectedComponentsWithStats((X > 20).astype(np.uint8), 8, cv2.CV_32S)
    stats = output[2]
    
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h
    
    X_fit = X[y1: y2, x1: x2]
    return X_fit, x1, y1  # Return the image and the offsets

# Read the CSV files
csv_path = '/Volumes/图图/VinDr/VinDr-Mammo/finding_annotations.csv'
split_csv_path = '../segdetdata/segdet_split.csv'
df = pd.read_csv(csv_path)
split_df = pd.read_csv(split_csv_path)

# Filter for VinDr-Mammo dataset
split_df = split_df[split_df['dataset'] == 'VinDr-Mammo']

# Only keep rows where 'finding_categories' contains 'Mass'
df['finding_categories'] = df['finding_categories'].apply(ast.literal_eval)
df = df[df['finding_categories'].apply(lambda x: 'Mass' in x)]

# Adjust bboxes after image has been fit
def adjust_bboxes(row, x_offset, y_offset):
    xmin = max(row['xmin'] - x_offset, 0)
    ymin = max(row['ymin'] - y_offset, 0)
    xmax = max(row['xmax'] - x_offset, 0)
    ymax = max(row['ymax'] - y_offset, 0)
    return [xmin, ymin, xmax, ymax]

base_path = '/Volumes/图图/VinDr/VinDr-Mammo/images'
target_path = '../segdetdata/VinDr-Mammo'

# Create output directories
for split in ['Train', 'Eval', 'Test']:
    os.makedirs(os.path.join(target_path, split), exist_ok=True)

for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_path = os.path.join(base_path, row['study_id'], row['image_id'] + '.dicom')
    
    if not os.path.exists(img_path):
        continue
    
    # Get split information from CSV
    split_info = split_df[split_df['data_name'] == row['image_id']]
    if split_info.empty:
        print(f"No split info found for {row['image_id']}")
        continue
    
    data_split = split_info['data_split'].values[0]
    
    # Process image and bboxes
    img, x_offset, y_offset = fit_image(img_path)
    
    bboxes = []
    for _, r in df[df['image_id'] == row['image_id']].iterrows():
        bbox = adjust_bboxes(r, x_offset, y_offset)
        bboxes.append(bbox)
    
    img_folder = os.path.join(target_path, data_split, row['image_id'])
    os.makedirs(img_folder, exist_ok=True)
    
    # Save image
    jpg_path = os.path.join(img_folder, 'img.jpg')
    cv2.imwrite(jpg_path, img)

    # Save bboxes
    npy_path = os.path.join(img_folder, 'bboxes.npy')
    np.save(npy_path, bboxes)

print("Processing complete.")

100%|██████████| 1226/1226 [08:03<00:00,  2.54it/s]

Processing complete.
